# [1075. Project Employees I](https://leetcode.com/problems/project-employees-i/description/?envType=study-plan-v2&envId=top-sql-50)

Table: Project

<pre>+-------------+---------+
| Column Name | Type    |
+-------------+---------+
| project_id  | int     |
| employee_id | int     |
+-------------+---------+</pre>
(project_id, employee_id) is the primary key of this table.
employee_id is a foreign key to Employee table.
Each row of this table indicates that the employee with employee_id is working on the project with project_id.


Table: Employee

<pre>+------------------+---------+
| Column Name      | Type    |
+------------------+---------+
| employee_id      | int     |
| name             | varchar |
| experience_years | int     |
+------------------+---------+</pre>
employee_id is the primary key of this table. It's guaranteed that experience_years is not NULL.
Each row of this table contains information about one employee.


Write an SQL query that reports the average experience years of all the employees for each project, rounded to 2 digits.

Return the result table in any order.

The query result format is in the following example.



Example 1:

Input:
Project table:
<pre>+-------------+-------------+
| project_id  | employee_id |
+-------------+-------------+
| 1           | 1           |
| 1           | 2           |
| 1           | 3           |
| 2           | 1           |
| 2           | 4           |
+-------------+-------------+</pre>
Employee table:
<pre>+-------------+--------+------------------+
| employee_id | name   | experience_years |
+-------------+--------+------------------+
| 1           | Khaled | 3                |
| 2           | Ali    | 2                |
| 3           | John   | 1                |
| 4           | Doe    | 2                |
+-------------+--------+------------------+</pre>
Output:
<pre>+-------------+---------------+
| project_id  | average_years |
+-------------+---------------+
| 1           | 2.00          |
| 2           | 2.50          |
+-------------+---------------+</pre>
Explanation: The average experience years for the first project is (3 + 2 + 1) / 3 = 2.00 and for the second project is (3 + 2) / 2 = 2.50

In [4]:
# pandas schema

import pandas as pd

data = [[1, 1], [1, 2], [1, 3], [2, 1], [2, 4]]
project = pd.DataFrame(data, columns=['project_id', 'employee_id']).astype(
    {'project_id': 'Int64', 'employee_id': 'Int64'})
data = [[1, 'Khaled', 3], [2, 'Ali', 2], [3, 'John', 1], [4, 'Doe', 2]]
employee = pd.DataFrame(data, columns=['employee_id', 'name', 'experience_years']).astype(
    {'employee_id': 'Int64', 'name': 'object', 'experience_years': 'Int64'})

# to pyspark dataframe

from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *

spark = SparkSession.builder.getOrCreate()

project_df = spark.createDataFrame(project)
employee_df = spark.createDataFrame(employee)

In [5]:
project_df.show(truncate=False)
employee_df.show(truncate=False)


+----------+-----------+
|project_id|employee_id|
+----------+-----------+
|1         |1          |
|1         |2          |
|1         |3          |
|2         |1          |
|2         |4          |
+----------+-----------+

+-----------+------+----------------+
|employee_id|name  |experience_years|
+-----------+------+----------------+
|1          |Khaled|3               |
|2          |Ali   |2               |
|3          |John  |1               |
|4          |Doe   |2               |
+-----------+------+----------------+



In [6]:
# solving in spark dataframe

project_df\
    .join(employee_df,['employee_id'],'inner')\
    .groupBy('project_id').agg(avg('experience_years').alias('average_years'))\
    .show()

+----------+-------------+
|project_id|average_years|
+----------+-------------+
|         1|          2.0|
|         2|          2.5|
+----------+-------------+



In [7]:
# solving in spark sql

employee_df.createOrReplaceTempView('employee')
project_df.createOrReplaceTempView('project')

spark.sql('''
select project_id, round(avg(experience_years),2) as average_years
from project
inner join employee
on project.employee_id=employee.employee_id
group by project_id;
''').show()

+----------+-------------+
|project_id|average_years|
+----------+-------------+
|         1|          2.0|
|         2|          2.5|
+----------+-------------+



In [ ]:
spark.stop()